## Remzijeva procena faze

Demonstracija praktične procene nepoznate faze korišćenjem jednokubitnih kvantnih kola.  

- Kreiraju se dva  nezavisna kvantna kola, svako sa početnim stanjem kubita u superpoziciji, na koje se zatim primenjuju $P(\varphi)$ operacije. Kubiti se zatim mere duž X i Y osa respektivno.  
- Iz statistika merenja se nalazi $\cos\varphi$ i $\sin\varphi$, a zatim izračunava $\varphi$ pomoću funkcije `arctan2`.  

![Ramsey-phase-estimation](images/Ramsey-phase-estimation.png)

### Zadatak
Pokrenuti program, uneti ciljnu vrednost ugla (npr. `3*pi/8`) i uporedi dobijeni ugao sa ciljnim.

### Očekivani rezultat
- Crteži kvantnih kola za merenje po X i Y osama.
- Vrednosti ciljnog ugla i izmerenog ugla.

### Eksperimentisanje
- Menjati vrednost `shots` i uočiti kako se menja tačnost rezultata.
- Eksperimentisati sa različitim uglovima, uključujući negativne i one veće od `2*pi`.


In [ ]:
from IPython.display import display

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
import numpy as np

pi = np.pi        # pi constant

# -- Input parameters --
fi = eval(input("Angle in radians (e.g., 3*pi/8): "))
shots = 2000      # Number of measurement shots

# -- circuit for measurement along the X-axis --
qcx = QuantumCircuit(1, 1)
qcx.h(0)          # prepare superposition |+>
qcx.p(fi, 0)      # apply phase gate P(φ)
qcx.h(0)          # rotate X-axis onto Z-axis for measurement
qcx.measure(0, 0)

# -- circuit for measurement along the Y-axis --
qcy = QuantumCircuit(1, 1)
qcy.h(0)          # prepare superposition |+>
qcy.p(fi, 0)      # apply phase gate P(φ)
qcy.sdg(0)        # rotate Y-axis onto X-axis (S† gate)
qcy.h(0)          # rotate X-axis onto Z-axis for measurement
qcy.measure(0, 0)

# -- run simulation for measurement along the X-axis --
simulator = AerSimulator()
result = simulator.run(qcx, shots=shots).result()
counts = result.get_counts(qcx)
cosfi = (counts['0']/shots) - (counts['1']/shots)  # estimate cos(φ)

# -- run simulation for measurement along the Y-axis --
result = simulator.run(qcy, shots=shots).result()
counts = result.get_counts(qcy)
sinfi = (counts['0']/shots) - (counts['1']/shots)  # estimate sin(φ)

display(circuit_drawer(qcx, style="bw", output="mpl"))
display(circuit_drawer(qcy, style="bw", output="mpl"))

print("Target angle:\t", fi)
print("Measured angle:\t", np.arctan2(sinfi, cosfi))
